# Deep Learning for tabular data using Pytorch

<b>Dataset</b> - https://www.kaggle.com/c/shelter-animal-outcomes

<b>Problem Statement</b>: Given certain features about a shelter animal (like age, sex, color, breed), predict its outcome.

There are 5 possible outcomes: Return_to_owner, Euthanasia, Adoption, Transfer, Died. We are expected to find the probability of an animal's outcome belonging to each of the 5 categories.

## Library imports

In [49]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from datetime import datetime

## Load Data

#### Training set 

In [50]:
train = pd.read_csv('train.csv')
print("Shape:", train.shape)
train.head()

Shape: (26729, 10)


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


#### Test set

In [51]:
test = pd.read_csv('test.csv')
print("Shape:", test.shape)
test.head()

Shape: (11456, 8)


,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
2,3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
3,4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
4,5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White


#### Sample submission file

For each row, each outcome's probability needs to be filled into the columns

In [4]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0


## Very basic data exploration

#### How balanced is the dataset?

Adoption and Transfer seem to occur a lot more than the rest

In [53]:
Counter(train['OutcomeType'])

Counter({'Return_to_owner': 4786,
         'Euthanasia': 1555,
         'Adoption': 10769,
         'Transfer': 9422,
         'Died': 197})

#### What are the most common names and how many times do they occur? 

There seem to be too many Nan values. Name might not be a very important factor too

In [54]:
Counter(train['Name']).most_common(5)

[(nan, 7691), ('Max', 136), ('Bella', 135), ('Charlie', 107), ('Daisy', 106)]

## Data preprocessing

OutcomeSubtype column seems to be of no use, so we drop it. Also, since animal ID is unique, it doesn't help in training

In [55]:
train_X = train.drop(columns= ['OutcomeType', 'OutcomeSubtype', 'AnimalID'])
Y = train['OutcomeType']
test_X = test

In [56]:
train_X.columns, Y.name

(Index(['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'AgeuponOutcome',
        'Breed', 'Color'],
       dtype='object'),
 'OutcomeType')

In [57]:
test_X.columns

Index(['ID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

#### Stacking train and test set so that they undergo the same preprocessing 

In [58]:
stacked_df = train_X.append(test_X.drop(columns=['ID']))

In [59]:
stacked_df.head(10)

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Hambone,2014-02-12 18:22:00,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Emily,2013-10-13 12:44:00,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Pearce,2015-01-31 12:28:00,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,NaN,2014-07-11 19:09:00,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,NaN,2013-11-15 12:52:00,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,Elsa,2014-04-25 13:04:00,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,Jimmy,2015-03-28 13:11:00,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,NaN,2015-04-30 17:02:00,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,Lucy,2014-02-04 17:17:00,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,NaN,2014-05-03 07:48:00,Dog,Spayed Female,1 year,Cairn Terrier,White


#### splitting datetime into month and year

In [60]:
# stacked_df['DateTime'] = pd.to_datetime(stacked_df['DateTime'])
# stacked_df['year'] = stacked_df['DateTime'].dt.year
# stacked_df['month'] = stacked_df['DateTime'].dt.month
stacked_df = stacked_df.drop(columns=['DateTime'])
stacked_df.head()

,Name,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Hambone,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Emily,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Pearce,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,NaN,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,NaN,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


#### dropping columns with too many nulls

In [61]:
for col in stacked_df.columns:
    if stacked_df[col].isnull().sum() > 10000:
        print("dropping", col, stacked_df[col].isnull().sum())
        stacked_df = stacked_df.drop(columns = [col])

dropping Name 10916


In [62]:
stacked_df.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [63]:
stacked_df.dtypes

AnimalType        object
SexuponOutcome    object
AgeuponOutcome    object
Breed             object
Color             object
dtype: object

#### label encoding

In [64]:
for col in stacked_df.columns:
    if stacked_df.dtypes[col] == "object":
        stacked_df[col] = stacked_df[col].fillna("NA")
    else:
        stacked_df[col] = stacked_df[col].fillna(0)
    stacked_df[col] = LabelEncoder().fit_transform(stacked_df[col])

In [65]:
stacked_df.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,3,5,1482,146
1,0,4,5,775,184
2,1,3,21,1293,97
3,0,1,26,775,47
4,1,3,21,1101,311


In [66]:
# making all variables categorical
for col in stacked_df.columns:
    stacked_df[col] = stacked_df[col].astype('category')

In [67]:
stacked_df.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,3,5,1482,146
1,0,4,5,775,184
2,1,3,21,1293,97
3,0,1,26,775,47
4,1,3,21,1101,311


In [68]:
stacked_df.dtypes

AnimalType        category
SexuponOutcome    category
AgeuponOutcome    category
Breed             category
Color             category
dtype: object

#### splitting back train and test

In [69]:
X = stacked_df[0:26729]
test_processed = stacked_df[26729:]

#check if shape[0] matches original
print("train shape: ", X.shape, "orignal: ", train.shape)
print("test shape: ", test_processed.shape, "original: ", test.shape)

train shape:  (26729, 5) orignal:  (26729, 10)
test shape:  (11456, 5) original:  (11456, 8)


#### Encoding target

In [70]:
Y = LabelEncoder().fit_transform(Y)

#sanity check to see numbers match and matching with previous counter to create target dictionary
print(Counter(train['OutcomeType']))
print(Counter(Y))
target_dict = {
    'Return_to_owner' : 3,
    'Euthanasia': 2,
    'Adoption': 0,
    'Transfer': 4,
    'Died': 1
}

Counter({'Adoption': 10769, 'Transfer': 9422, 'Return_to_owner': 4786, 'Euthanasia': 1555, 'Died': 197})
Counter({0: 10769, 4: 9422, 3: 4786, 2: 1555, 1: 197})


In [71]:
Y

array([3, 2, 0, ..., 0, 4, 4])

#### train-valid split

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.10, random_state=0)
X_train.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
6917,1,3,5,1293,146
13225,0,4,33,1515,231
2697,1,4,5,1353,43
21905,1,3,31,245,40
17071,0,4,37,775,156


In [73]:
X.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,3,5,1482,146
1,0,4,5,775,184
2,1,3,21,1293,97
3,0,1,26,775,47
4,1,3,21,1101,311


In [74]:
len(np.unique(np.array(list(X.Breed.cat.categories))))

1678

#### Choosing columns for embedding

In [75]:
#categorical embedding for columns having more than two values
embedded_cols = {n: len(col.cat.categories) for n,col in X.items() if len(col.cat.categories) > 2}
embedded_cols

{'SexuponOutcome': 6, 'AgeuponOutcome': 46, 'Breed': 1678, 'Color': 411}

In [76]:
embedded_col_names = embedded_cols.keys()
len(X.columns) - len(embedded_cols) #number of numerical columns

1

In [78]:
list(embedded_col_names)

['SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color']

#### Determining size of embedding 
(borrowed from https://www.usfca.edu/data-institute/certificates/fundamentals-deep-learning lesson 2)

In [39]:
embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
embedding_sizes

[(6, 3), (46, 23), (1678, 50), (411, 50)]

In [82]:
X.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,3,5,1482,146
1,0,4,5,775,184
2,1,3,21,1293,97
3,0,1,26,775,47
4,1,3,21,1101,311


In [85]:
X_train.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
6917,1,3,5,1293,146
13225,0,4,33,1515,231
2697,1,4,5,1353,43
21905,1,3,31,245,40
17071,0,4,37,775,156


## Pytorch Dataset

In [79]:
class ShelterOutcomeDataset(Dataset):
    def __init__(self, X, Y, embedded_col_names):
        X = X.copy()
        self.X1 = X.loc[:,embedded_col_names].copy().values.astype(np.int64) #categorical columns
        self.X2 = X.drop(columns=embedded_col_names).copy().values.astype(np.float32) #numerical columns
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

In [80]:
#creating train and valid datasets
train_ds = ShelterOutcomeDataset(X_train, y_train, embedded_col_names)
valid_ds = ShelterOutcomeDataset(X_val, y_val, embedded_col_names)

In [81]:
for i in range(5):
    x1, x2, y = train_ds[i]
    print(f'{x1}, {x2}, {y}')

[   3    5 1293  146], [1.], 0
[   4   33 1515  231], [0.], 0
[   4    5 1353   43], [1.], 0
[  3  31 245  40], [1.], 4
[  4  37 775 156], [0.], 0


## Making device (GPU/CPU) compatible 
(borrowed from https://jovian.ml/aakashns/04-feedforward-nn)

In order to make use of a GPU if available, we'll have to move our data and model to it.

In [86]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [87]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [88]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [89]:
device = get_default_device()
device

device(type='cpu')

## Model

(modified from https://www.usfca.edu/data-institute/certificates/fundamentals-deep-learning lesson 2)

In [92]:
class ShelterOutcomeModel(nn.Module):
    def __init__(self, embedding_sizes, n_cont):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings) #length of all embeddings combined
        self.n_emb, self.n_cont = n_emb, n_cont
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 200)
        
        print(f'n_emb: {n_emb}')
        
        
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, 5)
        self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
        

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [93]:
model = ShelterOutcomeModel(embedding_sizes, 1)
to_device(model, device)

n_emb: 126


ShelterOutcomeModel(
  (embeddings): ModuleList(
    (0): Embedding(6, 3)
    (1): Embedding(46, 23)
    (2): Embedding(1678, 50)
    (3): Embedding(411, 50)
  )
  (lin1): Linear(in_features=127, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=70, bias=True)
  (lin3): Linear(in_features=70, out_features=5, bias=True)
  (bn1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (drops): Dropout(p=0.3, inplace=False)
)

#### Optimizer

In [94]:
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

#### Training function

In [95]:
def train_model(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
        batch = y.shape[0]
        output = model(x1, x2)
        loss = F.cross_entropy(output, y)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

#### Evaluation function

In [96]:
def val_loss(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x1, x2, y in valid_dl:
        current_batch_size = y.shape[0]
        out = model(x1, x2)
        loss = F.cross_entropy(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.max(out, 1)[1]
        correct += (pred == y).float().sum().item()
    print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    return sum_loss/total, correct/total

In [97]:
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss = train_model(model, optim, train_dl)
        print("training loss: ", loss)
        val_loss(model, valid_dl)

## Training 

In [98]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size,shuffle=True)

In [99]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [35]:
train_loop(model, epochs=8, lr=0.05, wd=0.00001)

training loss:  1.2027205801462073
valid loss 0.942 and accuracy 0.609
training loss:  1.012246810026172
valid loss 0.904 and accuracy 0.608
training loss:  0.982185461055729
valid loss 0.909 and accuracy 0.626
training loss:  0.970147291746574
valid loss 0.881 and accuracy 0.636
training loss:  0.9620310133146533
valid loss 0.919 and accuracy 0.615
training loss:  0.9558281915941863
valid loss 0.869 and accuracy 0.639
training loss:  0.9427465338385062
valid loss 0.869 and accuracy 0.636
training loss:  0.9387064109341742
valid loss 0.870 and accuracy 0.632


## Test Output

In [100]:
test_ds = ShelterOutcomeDataset(test_processed, np.zeros(len(test_processed)), embedded_col_names)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [110]:
len(test_dl), len(test_ds)

(12, 11456)

In [104]:
preds = []
i = 0
with torch.no_grad():
    for x1,x2,y in test_dl:
        out = model(x1, x2)
        
        print(f'{i} -> {out}, {out.shape}')
    
        prob = F.softmax(out, dim=1)
        
        print(f'{i} -> {prob}, {prob.shape}')
        
        preds.append(prob)
        i += 1

0 -> tensor([[ 0.4628, -0.4033,  1.9195,  0.7988, -0.1443],
        [-0.1962,  0.4124,  0.0023,  0.5885, -0.1025],
        [-0.1958,  0.3385, -0.0413, -1.1158, -0.8060],
        ...,
        [-1.0272, -1.3066,  0.3952,  0.8826,  0.0574],
        [-0.4524,  0.9888,  0.1017,  0.0577, -1.4174],
        [-0.0031, -0.6380,  0.1142,  0.4576,  0.5545]]), torch.Size([1000, 5])
0 -> tensor([[0.1306, 0.0549, 0.5605, 0.1828, 0.0712],
        [0.1361, 0.2501, 0.1660, 0.2983, 0.1495],
        [0.2077, 0.3544, 0.2424, 0.0828, 0.1128],
        ...,
        [0.0641, 0.0484, 0.2656, 0.4324, 0.1895],
        [0.1110, 0.4689, 0.1931, 0.1848, 0.0423],
        [0.1670, 0.0885, 0.1878, 0.2648, 0.2918]]), torch.Size([1000, 5])
1 -> tensor([[ 0.1827, -0.3220,  1.0296,  0.4319,  0.1140],
        [ 0.8496,  0.3297,  0.4693, -0.1728,  0.7202],
        [ 0.2068,  0.8893, -0.2304,  0.0607, -0.6940],
        ...,
        [ 0.4216, -0.1498, -0.2174, -0.3426,  0.0804],
        [ 0.0263, -1.1903,  0.2462, -0.3023,  0.

In [106]:
final_probs = [item for sublist in preds for item in sublist]

In [109]:
print(f'final_probs -> ({len(final_probs)}, {len(final_probs[0])})')

final_probs -> (11456, 5)


In [107]:
final_probs

[tensor([0.1306, 0.0549, 0.5605, 0.1828, 0.0712]),
 tensor([0.1361, 0.2501, 0.1660, 0.2983, 0.1495]),
 tensor([0.2077, 0.3544, 0.2424, 0.0828, 0.1128]),
 tensor([0.1612, 0.2720, 0.1460, 0.2922, 0.1285]),
 tensor([0.2230, 0.1911, 0.2467, 0.2125, 0.1267]),
 tensor([0.0776, 0.2969, 0.1407, 0.1538, 0.3309]),
 tensor([0.1419, 0.1052, 0.1012, 0.2517, 0.4000]),
 tensor([0.1192, 0.3286, 0.3097, 0.1665, 0.0760]),
 tensor([0.2018, 0.2236, 0.1252, 0.1421, 0.3074]),
 tensor([0.2458, 0.1207, 0.1714, 0.2075, 0.2546]),
 tensor([0.0753, 0.1972, 0.4006, 0.2162, 0.1107]),
 tensor([0.1517, 0.2807, 0.0994, 0.2559, 0.2122]),
 tensor([0.3340, 0.0944, 0.2905, 0.1381, 0.1429]),
 tensor([0.1309, 0.1806, 0.1295, 0.0947, 0.4644]),
 tensor([0.1425, 0.2212, 0.2722, 0.1205, 0.2436]),
 tensor([0.2014, 0.2429, 0.1377, 0.2016, 0.2164]),
 tensor([0.1760, 0.5006, 0.1087, 0.1832, 0.0315]),
 tensor([0.1937, 0.2159, 0.1323, 0.3435, 0.1147]),
 tensor([0.1023, 0.1152, 0.1378, 0.3497, 0.2950]),
 tensor([0.2317, 0.1001, 0.2465

In [39]:
len(final_probs)

11456

In [40]:
target_dict

{'Return_to_owner': 3,
 'Euthanasia': 2,
 'Adoption': 0,
 'Transfer': 4,
 'Died': 1}

In [41]:
sample.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0


In [42]:
sample['Adoption'] = [float(t[0]) for t in final_probs]
sample['Died'] = [float(t[1]) for t in final_probs]
sample['Euthanasia'] = [float(t[2]) for t in final_probs]
sample['Return_to_owner'] = [float(t[3]) for t in final_probs]
sample['Transfer'] = [float(t[4]) for t in final_probs]
sample.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,0.134578,0.007721,0.075624,0.200941,0.581136
1,2,0.516681,0.001678,0.019207,0.296993,0.165442
2,3,0.432114,0.004433,0.035524,0.169191,0.358738
3,4,0.084035,0.008793,0.060623,0.101076,0.745472
4,5,0.529311,0.001309,0.012651,0.285051,0.171678


In [43]:
sample.to_csv('samp.csv', index=False)

In [44]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit()

[jovian] Saving notebook..


<IPython.core.display.Javascript object>